In [42]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/forcoding247/Data-Science_Project.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = 'forcoding247'
os.environ['MLFLOW_TRACKING_PASSWORD'] = "f8a0e72116c9a3692360c53afb2c4baaed03b176"

In [43]:
import os
%pwd

'd:\\MLOPs\\Data-Science_Project'

In [44]:
os.chdir("d:\\MLOPs\\Data-Science_Project")   

In [45]:
%pwd

'd:\\MLOPs\\Data-Science_Project'

In [46]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [47]:
from src.datacience.constants import *
from src.datacience.utils.common import read_yaml, create_directories, save_json

In [48]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluator
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluator_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            all_params = params,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/forcoding247/Data-Science_Project.mlflow"
        )

        return model_evaluator_config

In [49]:
import os 
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [50]:
class ModelEvaluator:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config

    def  eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rmse, mae, r2
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_store_type = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)


            ## Saving metrics as local 
            scores = { "rmse" : rmse, "mae" : mae, "r2": r2}
            save_json(path = Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric('r2', r2)

            ## Model Registry does not work with file store
            if tracking_url_store_type != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name='ElasticNetModel')

            else:
                mlflow.sklearn.log_model(model, "model")

In [51]:
print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: https://dagshub.com/forcoding247/Data-Science_Project.mlflow


In [52]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluator(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-06-07 13:47:56,637 : INFO : common : Yaml file config\config.yaml loaded successfully.]
[2025-06-07 13:47:56,639 : INFO : common : Yaml file params.yaml loaded successfully.]
[2025-06-07 13:47:56,641 : INFO : common : Yaml file schema.yaml loaded successfully.]
[2025-06-07 13:47:56,643 : INFO : common : Created Directory at : artifacts]
[2025-06-07 13:47:56,644 : INFO : common : Created Directory at : artifacts/model_evaluation]
[2025-06-07 13:47:58,323 : INFO : common : json file saved at : artifacts\model_evaluation\metrics.json]


2025/06/07 13:48:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/06/07 13:48:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run painted-goat-899 at: https://dagshub.com/forcoding247/Data-Science_Project.mlflow/#/experiments/0/runs/b3ea075dc61e4c7eba89fae9d030419c
🧪 View experiment at: https://dagshub.com/forcoding247/Data-Science_Project.mlflow/#/experiments/0
